# BMI ML Bootcamp #3 - Basic Neural Net Implementation with MNIST

Adapted pretty heavily from BMI 219 HW 1 - shout out to Kangway

PyTorch tutorial - https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

The pytorch documentation can be a bit rough. If you're having trouble, feel free to ask me (or Scott or Erin of course :) ) - this tutorial might be difficult if you're unfamiliar with NNs/pytorch.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image

For this tutorial, we'll be using the MNIST dataset, consisting of handwritted numbers and their accompanying label. The first thing to do is download, load, and normalize this dataset. Running the cell below will create a 'data/MNIST' file containing the data. We also create a preprocessing transform to normalize the mean and standard deviation of training set to be 0.1307 and 0.3081 respectively. 

In [ ]:
preprocessing = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(
    './data', train=True, download=True,
    transform=preprocessing)
                               
test_dataset = datasets.MNIST(
    './data',train=False, download=True, 
    transform=preprocessing)

How many training examples are there? How many examples in the test set? What are the image dimensions? Where can you find the corresponding labels for each training example?

Split the training data further into training and validation sets - this well help us to prevent overfitting. What split did you choose?

In [ ]:
train_set, val_set = torch.utils.data.random_split(train_dataset, ?)

In [ ]:
# This cell plots a few random training examples along with the labels, always good to sanity check your data
fig, axes = plt.subplots(1, 4, figsize=(20,5))

for ax_index, i in enumerate(np.random.choice(list(range(train_set.dataset.data.shape[0])), 4)):
    x, _ = train_set.dataset[i] # x is now a torch.Tensor
    axes[ax_index].imshow(x.numpy()[0], cmap='gray')
    axes[ax_index].set_title(train_set.dataset.targets[i].item())

## Set up dataloaders - one for training, one for validation, and one for testing

In [ ]:
BATCH_SIZE = 128  ### Please change this as necessary
NUM_WORKERS = 1  ### Use more workers for more CPU threads

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers=NUM_WORKERS)

val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers=NUM_WORKERS)

# Set up test_loader here, same as the training loader
test_loader = ?

## Define the neural network architecture

I implemented SmallNet with 2 2dconv/max pool layers, followed by a few fully connected layers, but you can do this however you would like. 

In [ ]:
class SmallNet(nn.Module):
    def __init__(self):
        super(SmallNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, 3) # 1 input image channel, 1 output channels, 3x3 square convolution
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(1, 1, 3) # 1 input image channel, 1 output channels, 3x3 square convolution
        self.fc1 = nn.Linear(1*5*5, 15)
        self.fc2 = nn.Linear(15, 10) #10 possibilities (0 - 9), so we want 10 output nodes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,1*5*5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1) # Mutually exclusive classification task
        return output

## Make your own architecture here - go ham

In [ ]:
class YourNet(nn.Module):
    def __init__(self):
        super(YourNet, self).__init__()

        #add your layers here
        pass
    
    def forward(self, x):
        # Add your forward pass here - choose whatever activation functions you think are best
        pass

## Define your training function here

Your training function should accomplish a few things, namely
* iterate through n epochs
* load batches
* feed forward
* calculate loss
* backpropagation
* return the losses

I've included my training function header and skeleton.

In [ ]:
def train(train_loader, val_loader, model, optimizer, criterion, device,
          n_epochs=10, log_interval = 1, save = False, prefix = 'test', **kwargs):
    
    model.train()
    losses = []
    val_losses = []
    
    for epoch in range(n_epochs + 1):
        #training loss
        epoch_losses = []
        val_epoch_losses = []
        for batch_idx, examples in enumerate(train_loader):
            # Get values and lables from train_loader, send to device
            values, labels = examples 
            values = values.to(device, dtype=torch.float) #train on set device (I used GPU) 
            labels = labels.to(device)

            optimizer.zero_grad() #zero gradients

            ? #feed forward
            ? #get loss
            ? #back propagation

            optimizer.step()

            epoch_losses.append(loss.item())
        
        #validation loss - don't update gradients
        with torch.no_grad():
            for batch_idx, examples in enumerate(val_loader):
                ? # Get values and labels from val_loader, send to device
                
                ? #feed forward
                ? #get loss 
                
                val_epoch_losses.append(loss.item())
            
        ? # Update losses and val_losses
    
        if epoch % log_interval == 0:
            print('Train Epoch: {} \tLoss: {:.6f}, Val: {:.6f}'.format(
                epoch, losses[epoch], val_losses[epoch]))
        
            #save temporary model
            if save:
                state = {'epoch': epoch, 'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict()}
                torch.save(state, f'{prefix}-{epoch}.pth')

    return losses, val_losses

In [ ]:
#check if cuda is available - if you're running this on your laptop, it probably won't be
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Call your training function and record the losses

In [ ]:
# These are the hyper-parameters that I used, but feel free to mess around
lr = 0.001 
n_epochs = 50
log_interval = n_epochs / 10

model = SmallNet() # Change to your model, once you've decided on an architecture
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=lr) # Don't have to use Adam - checkout SGD etc.
criterion = nn.CrossEntropyLoss() # Why are we using cross entropy loss here? Could we be using a different loss function?

losses = train(train_loader, val_loader, model, optimizer, criterion, device, 
               n_epochs = n_epochs, log_interval = log_interval, 
               save = False, prefix = f'SmallNet')

## Plot the validation and loss curves

Plotting the loss curves helps us to see if our model is still training, and if the loss at each epoch looks reasonable. By including a validation set, we are able to observe if our model is overfitting, indicated by a validation loss much higher than the training loss. When you plot your training and validation losses, do you observe anything that looks like overfitting?

In [ ]:
plt.plot(losses[0], label = "Training")
plt.plot(losses[1], label = "Validation")
plt.legend()

## Evaluate the performance of the model using the test set

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)
        outputs = model.forward(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels)
        for i in range(c.shape[0]):
            label = labels[i].item()
            class_correct[label] += c[i].item()
            class_total[label] += 1

correct = []
for i in range(10):
    correct.append(class_correct[i] / class_total[i])

## Visualize a few test examples and their labels

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20,5))

mini_batch, labels = next(iter(test_loader))
with torch.no_grad():
    outputs = model.forward(mini_batch)
for ax_index, i in enumerate(zip(mini_batch[:4], labels[:4], outputs[:4])):
    axes[ax_index].imshow(i[0].numpy()[0], cmap='gray')
    axes[ax_index].set_title(f'Predicted: {torch.argmax(i[2]).item()}, Actual: {i[1].item()}')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
plt.bar(np.arange(10), correct)

## Optional: Train an autoencoder, either on the MNIST dataset or the CelebA Faces dataset

Unlike the classification problem above, the goal of an autoencoder is to compress input data to a lower dimensional representation (encoding layers), followed by decompressing to recreate the initial input (decoding layers). This means that the dimensions of the output layer will need to change, as well as the loss function. For example, a typical architecture for MNIST would be [784, 392, 196, 98, 2, 98, 196, 392, 784] (28 * 28 * 1 = 784, for the input size). How would you modify SmallNet or your classifier architecture to approach this problem?

If you're interested in doing this, read https://www.cs.toronto.edu/~hinton/science.pdf "Reducing the dimensionality of data with neural networks". They discuss their approach and architecture.